In [1]:
# Helper libraries
from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
import os
from time import time
from sklearn.utils import class_weight

# tensorflow and keras
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Dense
from keras.backend.tensorflow_backend import set_session
from keras import metrics
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

# models
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# Data utils
from keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

1.13.1


Using TensorFlow backend.


In [2]:
# gpu stuff
%env CUDA_VISIBLE_DEVICES=0,1
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

env: CUDA_VISIBLE_DEVICES=0,1


In [3]:
# Seeds
tf.random.set_random_seed(123)
np.random.seed(123)

In [4]:
# Hyperparameters
batch_size = 10
train_epochs = 200
architecture = 'mobilenetv2'
weighted_loss = False

In [5]:
#Data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20, 
                                   zoom_range=0.15,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'data/val',
        target_size=(224,224),
        batch_size=1,
        class_mode='categorical')

Found 22794 images belonging to 9 classes.
Found 2537 images belonging to 9 classes.


In [6]:
N_train = train_generator.n
N_val = validation_generator.n
class_names = sorted(list(train_generator.class_indices.keys()))
N_classes = len(class_names)

In [7]:
N_train,N_val,class_names

(22794, 2537, ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'UNK', 'VASC'])

In [8]:
# Weighted loss - not useful here
if(weighted_loss):
    Y_train = train_generator.classes
    class_weight = class_weight.compute_class_weight('balanced',np.unique(Y_train),Y_train)
    class_weight = dict(enumerate(class_weight))
    print(class_weight)
else:
    class_weight = None

In [9]:
if(architecture=='mobilenetv2'):
    net = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3), pooling='avg', classes=N_classes)
elif(architecture=='inception_resnet'):
    net = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling='avg', classes=N_classes)
    
x = net.layers[-1].output
fc = Dense(N_classes,activation='softmax')(x)
model = Model(inputs=net.input, outputs=fc)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
show_summary = False
if(show_summary):
    model.summary()

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
name = architecture+('_weighted')*weighted_loss
checkpointer = ModelCheckpoint(filepath=os.path.join("models",name+"_weights.hdf5"), 
                               monitor = 'val_acc',
                               verbose=1, 
                               save_best_only=True)

In [12]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=N_train//train_generator.batch_size,
        epochs=train_epochs,
        validation_data=validation_generator,
        validation_steps=N_val//validation_generator.batch_size,
        class_weight = class_weight,
        verbose= 1,
        shuffle = True,
        callbacks= [tensorboard,checkpointer])

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
2279/2279 [==============================] - 913s 401ms/step - loss: 1.1133 - acc: 0.6099 - val_loss: 2.3859 - val_acc: 0.5518

Epoch 00001: val_acc improved from -inf to 0.55183, saving model to models/mobilenetv2_weights.hdf5
Epoch 2/200
2279/2279 [==============================] - 890s 391ms/step - loss: 1.0144 - acc: 0.6381 - val_loss: 1.5794 - val_acc: 0.4407

Epoch 00002: val_acc did not improve from 0.55183
Epoch 3/200
2279/2279 [==============================] - 911s 400ms/step - loss: 0.9776 - acc: 0.6518 - val_loss: 1.9442 - val_acc: 0.4923

Epoch 00003: val_acc did not improve from 0.55183
Epoch 4/200
2279/2279 [==============================] - 889s 390ms/step - loss: 0.9532 - acc: 0.6569 - val_loss: 2.7272 - val_acc: 0.3780

Epoch 00004: val_acc did not improve from 0.55183
Epoch 5/200
2279/2279 [==============================] - 888s 390ms/step - loss: 0.9244 - acc: 0.6661 - val_loss: 1.5911 - val_acc: 0.6023

Ep

2279/2279 [==============================] - 943s 414ms/step - loss: 0.5309 - acc: 0.8044 - val_loss: 0.7825 - val_acc: 0.7229

Epoch 00040: val_acc did not improve from 0.74616
Epoch 41/200
2279/2279 [==============================] - 966s 424ms/step - loss: 0.5224 - acc: 0.8088 - val_loss: 0.8416 - val_acc: 0.7458

Epoch 00041: val_acc did not improve from 0.74616
Epoch 42/200
2279/2279 [==============================] - 949s 416ms/step - loss: 0.5110 - acc: 0.8106 - val_loss: 1.0447 - val_acc: 0.7190

Epoch 00042: val_acc did not improve from 0.74616
Epoch 43/200
2279/2279 [==============================] - 950s 417ms/step - loss: 0.5129 - acc: 0.8104 - val_loss: 0.7819 - val_acc: 0.7288

Epoch 00043: val_acc did not improve from 0.74616
Epoch 44/200
2279/2279 [==============================] - 952s 418ms/step - loss: 0.5107 - acc: 0.8133 - val_loss: 1.0121 - val_acc: 0.6480

Epoch 00044: val_acc did not improve from 0.74616
Epoch 45/200
2279/2279 [==============================] - 


Epoch 00081: val_acc did not improve from 0.77769
Epoch 82/200
2279/2279 [==============================] - 934s 410ms/step - loss: 0.3683 - acc: 0.8633 - val_loss: 0.8046 - val_acc: 0.7745

Epoch 00082: val_acc did not improve from 0.77769
Epoch 83/200
2279/2279 [==============================] - 926s 406ms/step - loss: 0.3587 - acc: 0.8650 - val_loss: 0.8181 - val_acc: 0.7525

Epoch 00083: val_acc did not improve from 0.77769
Epoch 84/200
2279/2279 [==============================] - 918s 403ms/step - loss: 0.3545 - acc: 0.8710 - val_loss: 0.8019 - val_acc: 0.7607

Epoch 00084: val_acc did not improve from 0.77769
Epoch 85/200
2279/2279 [==============================] - 937s 411ms/step - loss: 0.3532 - acc: 0.8732 - val_loss: 0.8658 - val_acc: 0.7544

Epoch 00085: val_acc did not improve from 0.77769
Epoch 86/200
2279/2279 [==============================] - 932s 409ms/step - loss: 0.3476 - acc: 0.8699 - val_loss: 0.8040 - val_acc: 0.7741

Epoch 00086: val_acc did not improve from 0.

2279/2279 [==============================] - 897s 394ms/step - loss: 0.2577 - acc: 0.9053 - val_loss: 0.8846 - val_acc: 0.7718

Epoch 00124: val_acc did not improve from 0.78597
Epoch 125/200
2279/2279 [==============================] - 896s 393ms/step - loss: 0.2571 - acc: 0.9065 - val_loss: 0.7787 - val_acc: 0.7793

Epoch 00125: val_acc did not improve from 0.78597
Epoch 126/200
2279/2279 [==============================] - 890s 391ms/step - loss: 0.2546 - acc: 0.9061 - val_loss: 0.7893 - val_acc: 0.7745

Epoch 00126: val_acc did not improve from 0.78597
Epoch 127/200
2279/2279 [==============================] - 917s 403ms/step - loss: 0.2538 - acc: 0.9050 - val_loss: 1.0589 - val_acc: 0.7339

Epoch 00127: val_acc did not improve from 0.78597
Epoch 128/200
2279/2279 [==============================] - 914s 401ms/step - loss: 0.2529 - acc: 0.9054 - val_loss: 1.0617 - val_acc: 0.7406

Epoch 00128: val_acc did not improve from 0.78597
Epoch 129/200
2279/2279 [=============================

2279/2279 [==============================] - 956s 420ms/step - loss: 0.1959 - acc: 0.9273 - val_loss: 1.1806 - val_acc: 0.7387

Epoch 00166: val_acc did not improve from 0.81514
Epoch 167/200
2279/2279 [==============================] - 949s 416ms/step - loss: 0.1889 - acc: 0.9305 - val_loss: 1.0228 - val_acc: 0.7568

Epoch 00167: val_acc did not improve from 0.81514
Epoch 168/200
2279/2279 [==============================] - 944s 414ms/step - loss: 0.1894 - acc: 0.9317 - val_loss: 0.8075 - val_acc: 0.7868

Epoch 00168: val_acc did not improve from 0.81514
Epoch 169/200
2279/2279 [==============================] - 946s 415ms/step - loss: 0.1923 - acc: 0.9275 - val_loss: 0.7752 - val_acc: 0.7954

Epoch 00169: val_acc did not improve from 0.81514
Epoch 170/200
2279/2279 [==============================] - 921s 404ms/step - loss: 0.1929 - acc: 0.9301 - val_loss: 0.8948 - val_acc: 0.7670

Epoch 00170: val_acc did not improve from 0.81514
Epoch 171/200
2279/2279 [=============================

In [13]:
with open('models/'+name+'_architecture.json', 'w') as f:
    f.write(model.to_json())